### Adnan Altukleh, Abdulkarim Dawalibi
### Course DV2627

### Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
from scipy.stats import skew, kurtosis
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import  roc_auc_score

### Reading the data

In [3]:
train_oper=pd.read_csv("train_operational_readouts.csv")
train_tte=pd.read_csv("train_tte.csv")

### Labeling the data

In [4]:
# function to calculate the time to event
def set_class_label(row):
    if row['in_study_repair'] == 0:
        return 0
    elif 0 <= row['time'] <= 6:
        return 4
    elif 6 <= row['time'] <= 12:   
        return 3
    elif 12 <= row['time'] <= 24:
        return 2
    elif 24 <= row['time'] <= 48:
        return 1
    elif row['time'] > 48:
        return 1
    else:
        raise ValueError("Invalid time value: {}".format(row['time']))

In [5]:
# Get the last row for each vehicle_id in train_oper
last_rows = train_oper.groupby('vehicle_id').tail(1)

# Merge train_tte with last_rows on vehicle_id
train_oper_label = pd.merge(last_rows, train_tte, on='vehicle_id')

# Calculate the difference between time_step and length_of_study_time_step
train_oper_label['time'] = train_oper_label['length_of_study_time_step'] - train_oper_label['time_step']
train_oper_label

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,...,397_29,397_30,397_31,397_32,397_33,397_34,397_35,length_of_study_time_step,in_study_repair,time
0,0,507.4,10189950.0,372685.0,NaN,41670.0,NaN,NaN,NaN,NaN,...,367.0,36431.0,3534852.0,4290245.0,1027186.0,138939.0,792.0,510.0,0,2.6
1,2,281.0,5648790.0,289371.0,233978571.0,68717.0,10415.0,9137870.0,74655621.0,45991626.0,...,246.0,18151.0,1193941.0,1680728.0,328910.0,57511.0,793.0,281.8,0,0.8
2,3,291.2,7603590.0,230831.0,273813365.0,100121.0,5918.0,8225139.0,17004223.0,10504195.0,...,204.0,7952.0,719693.0,2648967.0,531790.0,76915.0,712.0,293.4,0,2.2
3,4,203.0,4842780.0,210381.0,174662976.0,152385.0,7128.0,4342398.0,13348382.0,11538870.0,...,742.0,10529.0,370756.0,1651890.0,1282530.0,453350.0,1962.0,210.0,0,7.0
4,5,357.6,6623040.0,280531.0,246994778.0,164673.0,NaN,NaN,NaN,NaN,...,2601.0,9754.0,465567.0,1684524.0,979654.0,515597.0,8663.0,360.4,0,2.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23545,33639,112.6,2894100.0,75859.0,98975802.0,10040.0,6631.0,5229663.0,10519563.0,8797533.0,...,156.0,7958.0,239587.0,1024221.0,384568.0,101945.0,401.0,137.4,0,24.8
23546,33640,123.2,3925455.0,138179.0,139017335.0,35726.0,15397.0,5757292.0,11640261.0,7568543.0,...,556.0,26526.0,409625.0,1390447.0,474318.0,338311.0,5127.0,124.6,0,1.4
23547,33641,116.0,2567085.0,118371.0,91726867.0,9122.0,6783.0,3383972.0,6551850.0,6232055.0,...,100.0,25511.0,796852.0,1395692.0,264116.0,33488.0,222.0,123.0,0,7.0
23548,33642,126.4,3350700.0,80078.0,118733863.0,11251.0,755.0,9392926.0,16152271.0,7956861.0,...,80.0,16320.0,910530.0,1851490.0,290285.0,41111.0,536.0,126.6,0,0.2


In [6]:
train_oper_label['class_label'] = train_oper_label.apply(set_class_label, axis=1)

In [7]:
tt=train_oper_label.drop(columns=train_oper_label.columns[1:-1],axis=1)
training_data=pd.merge(tt,train_oper,on='vehicle_id')

In [8]:
training_data

,vehicle_id,class_label,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,...,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,0,0,11.2,167985.0,10787.0,7413813.0,2296.0,4110.0,1296420.0,1628265.0,...,95728.0,15609.0,1984.0,8.0,784.0,150228.0,261904.0,93172.0,17874.0,452.0
1,0,0,11.4,167985.0,10787.0,7413813.0,2296.0,4111.0,1302855.0,1628265.0,...,95729.0,15610.0,1984.0,8.0,784.0,150228.0,261905.0,93172.0,17874.0,452.0
2,0,0,19.6,331635.0,14525.0,13683604.0,2600.0,NaN,NaN,NaN,...,142900.0,19263.0,2441.0,12.0,1420.0,204832.0,313485.0,106464.0,19306.0,452.0
3,0,0,20.2,354975.0,15015.0,14540449.0,2616.0,NaN,NaN,NaN,...,150565.0,19832.0,2522.0,12.0,1444.0,211688.0,318901.0,107745.0,19406.0,453.0
4,0,0,21.0,365550.0,15295.0,14966985.0,2720.0,NaN,NaN,NaN,...,155913.0,20573.0,2562.0,12.0,1445.0,213956.0,323997.0,109514.0,19535.0,454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122447,33643,0,101.0,2136810.0,41412.0,81068654.0,10365.0,843.0,6408272.0,10964376.0,...,727848.0,34448.0,6540.0,36.0,19500.0,612343.0,626033.0,100155.0,17033.0,24.0
1122448,33643,0,107.0,2283285.0,44660.0,86639835.0,12317.0,843.0,6596477.0,12358026.0,...,780576.0,37488.0,6964.0,40.0,20484.0,652688.0,670517.0,107367.0,18901.0,24.0
1122449,33643,0,113.8,2462865.0,48370.0,93439419.0,13910.0,843.0,6669542.0,12830421.0,...,837953.0,40050.0,7524.0,44.0,21688.0,698824.0,722453.0,115851.0,21237.0,28.0
1122450,33643,0,119.8,2627250.0,51101.0,99682931.0,14854.0,843.0,7074468.0,13604706.0,...,898070.0,42866.0,7992.0,48.0,22732.0,757856.0,777630.0,123155.0,22357.0,32.0


### Selecting specific number of vehicles form each class

In [9]:
# selecting the vehicles from each class with complete data and equal number of vehicles from each class
class_1=training_data.loc[(training_data["class_label"]==1)]["vehicle_id"].unique()
class_2=training_data.loc[(training_data["class_label"]==2)& (training_data["vehicle_id"].isin(training_data.groupby("vehicle_id").filter(lambda x: x.notnull().all().all())["vehicle_id"].unique()))]["vehicle_id"].unique()[:len(class_1)]
class_3=training_data.loc[(training_data["class_label"]==3)& (training_data["vehicle_id"].isin(training_data.groupby("vehicle_id").filter(lambda x: x.notnull().all().all())["vehicle_id"].unique()))]["vehicle_id"].unique()[:len(class_1)]
class_4=training_data.loc[(training_data["class_label"]==4)& (training_data["vehicle_id"].isin(training_data.groupby("vehicle_id").filter(lambda x: x.notnull().all().all())["vehicle_id"].unique()))]["vehicle_id"].unique()[:len(class_1)]


In [10]:
# selecting all vehicles from class 0 with complete data
class_0=training_data.loc[(training_data["class_label"]==0)& (training_data["vehicle_id"].isin(training_data.groupby("vehicle_id").filter(lambda x: x.notnull().all().all())["vehicle_id"].unique()))]["vehicle_id"].unique()

In [11]:
df_1 = training_data.loc[(training_data["vehicle_id"].isin(class_1))]
df_2 = training_data.loc[(training_data["vehicle_id"].isin(class_2))]
df_3 = training_data.loc[(training_data["vehicle_id"].isin(class_3))]
df_4 = training_data.loc[(training_data["vehicle_id"].isin(class_4))]
df_0 = training_data.loc[(training_data["vehicle_id"].isin(class_0))]
data=pd.concat([df_0,df_1,df_2,df_3,df_4])
data

,vehicle_id,class_label,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,...,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
172,2,0,1.4,65520.0,2226.0,2891746.0,424.0,0.0,811605.0,1644061.0,...,22516.0,1524.0,232.0,4.0,337.0,12089.0,10388.0,2413.0,1081.0,88.0
173,2,0,1.6,73215.0,2401.0,3221614.0,488.0,0.0,824430.0,1845031.0,...,24450.0,1661.0,256.0,4.0,377.0,13125.0,11100.0,2609.0,1110.0,92.0
174,2,0,6.0,156150.0,5656.0,6727292.0,944.0,2776.0,1051575.0,3685576.0,...,62722.0,3778.0,648.0,16.0,873.0,33321.0,29052.0,6805.0,2379.0,96.0
175,2,0,50.8,1091040.0,73703.0,46242922.0,18216.0,5912.0,1935420.0,14616121.0,...,470495.0,33330.0,6527.0,128.0,4262.0,232677.0,345040.0,85935.0,22203.0,449.0
176,2,0,63.4,1310445.0,92428.0,55111815.0,20616.0,5912.0,2314425.0,18313081.0,...,558799.0,39695.0,7932.0,156.0,4879.0,274641.0,432085.0,109379.0,28767.0,593.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42277,779,4,230.6,6720645.0,200720.0,229802519.0,40972.0,1347.0,6972803.0,16410266.0,...,2491172.0,111713.0,21575.0,76.0,17609.0,869318.0,3313694.0,835877.0,119079.0,126.0
42278,779,4,236.6,6959310.0,207700.0,238055093.0,41924.0,1348.0,7218953.0,16903031.0,...,2577153.0,116069.0,22655.0,80.0,18393.0,891074.0,3421491.0,863341.0,122783.0,138.0
42279,779,4,242.8,7167750.0,214630.0,245294876.0,42908.0,1348.0,7408838.0,17315786.0,...,2653302.0,119911.0,23460.0,88.0,19073.0,910523.0,3516503.0,888233.0,125751.0,150.0
42280,779,4,248.8,7366410.0,220993.0,252324756.0,44836.0,1348.0,7622183.0,17724161.0,...,2739474.0,124667.0,24321.0,88.0,19985.0,936516.0,3638331.0,927205.0,131840.0,154.0


### Converting to Power spectral density (PSD)

In [12]:
# function that estimates the frequency of the signal
def estimate_fs(group):
    if len(group) < 2:
        return np.nan
    time_diffs = group['time_step'].diff().dropna()
    mean_diff = time_diffs.mean()
    if mean_diff == 0:
        return np.nan
    return 1 / mean_diff

In [13]:
# function that computes the power spectral density
def compute_psd(group):
    # Estimate the sampling frequency for the group
    fs = estimate_fs(group)
       
    psd_features = {}
    # Columns to skip
    columns_to_skip = ['vehicle_id', 'class_label', 'time_step']
    
    # Iterate over columns that are not in columns_to_skip
    for column in [col for col in group.columns if col not in columns_to_skip]:
        # Calculate the PSD using Welch's method
        f, Pxx_den = welch(group[column], fs=fs)
        psd_features[column] = Pxx_den
        
    # Convert the PSD features dictionary to a DataFrame
    psd_df = pd.DataFrame(psd_features)
    
    # Add the 'vehicle_id' back into the DataFrame if it's a groupby operation
    if 'vehicle_id' in group.columns:
        psd_df['vehicle_id'] = group['vehicle_id'].iloc[0]
    
    
    return psd_df

In [14]:
grouped = data.groupby('vehicle_id')
grouped_psd_dfs = grouped.apply(compute_psd)
psd_combined_df = grouped_psd_dfs.reset_index(drop=True)

In [15]:
# label the data
label=train_oper_label.drop(columns=train_oper_label.columns[1:-1],axis=1)
psd_combined_df=pd.merge(label,psd_combined_df,on='vehicle_id')
psd_combined_df

,vehicle_id,class_label,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,...,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,2,0,1.801266e+13,8.311010e+10,3.271568e+16,3.222816e+09,2.508907e+05,1.948458e+13,4.693872e+15,1.786114e+15,...,3.103526e+12,1.987722e+10,6.102489e+08,106780.731891,1.333921e+08,9.774308e+11,2.064357e+12,1.198911e+11,4.901280e+09,773725.677625
1,2,0,1.368028e+14,3.360947e+11,2.362335e+17,1.444445e+10,1.474122e+08,2.787171e+14,2.281714e+16,8.848792e+15,...,2.379530e+13,1.027913e+11,2.012062e+09,85712.367592,1.312519e+09,6.832272e+12,1.211736e+13,3.960113e+11,7.729815e+09,802576.105324
2,2,0,5.985169e+12,1.305118e+10,1.028956e+16,7.871946e+08,7.046216e+07,3.949908e+12,4.554220e+14,3.102671e+14,...,8.728714e+11,3.449150e+09,4.975657e+07,454.096609,7.049855e+07,3.589297e+11,5.813196e+11,1.255181e+10,6.134489e+07,37458.526648
3,2,0,1.472598e+11,8.890956e+08,2.570168e+14,1.398372e+08,3.199051e+07,3.771839e+12,1.787946e+14,2.475302e+13,...,9.055342e+10,3.385577e+08,8.779791e+06,825.599235,2.594179e+06,6.628615e+09,5.238378e+10,1.386922e+09,1.855831e+07,21412.039785
4,2,0,4.223230e+11,1.274794e+09,7.235092e+14,1.556801e+08,1.786432e+07,2.012687e+11,1.668541e+13,3.045308e+13,...,1.196348e+11,2.795104e+08,2.049627e+06,475.036782,2.411408e+06,1.190469e+10,6.331151e+10,1.479111e+09,9.713297e+06,1217.583597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331420,33643,0,9.792961e+09,4.089365e+06,1.347692e+13,5.809572e+04,4.358281e+04,1.944025e+11,3.042675e+11,5.119733e+10,...,1.484380e+09,3.527540e+06,3.448834e+04,0.188332,2.559452e+05,1.318536e+09,1.428300e+09,3.605492e+07,7.961996e+05,15.951897
331421,33643,0,5.239445e+09,2.487386e+06,6.858943e+12,6.866925e+04,2.524976e+04,2.783225e+11,3.069148e+11,8.353559e+10,...,8.349910e+08,1.506913e+06,2.471021e+03,3.286981,5.589331e+05,7.144842e+08,6.239829e+08,2.129606e+07,7.393060e+05,12.815349
331422,33643,0,3.255898e+09,1.505509e+06,4.716921e+12,5.645569e+03,5.188152e+04,2.892596e+11,1.529693e+11,5.286496e+10,...,3.046415e+08,9.382525e+05,1.775266e+04,9.692848,5.506107e+05,6.842541e+08,5.682599e+08,2.704650e+07,1.263130e+06,11.271328
331423,33643,0,3.385359e+09,1.537876e+06,5.669743e+12,3.307810e+04,9.027057e+04,2.199168e+11,2.367430e+11,1.181034e+10,...,3.061152e+08,1.826820e+06,2.772618e+04,4.801541,5.158491e+05,6.473885e+08,5.666977e+08,2.532595e+07,5.394936e+05,17.197900


### Applying feature Extraction

In [16]:
def calculate_spectral_features(signal, fs=1.0):
    f, Pxx = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    psd_norm = Pxx / np.sum(Pxx)
    spectral_entropy = entropy(psd_norm)
    spectral_kurtosis = kurtosis(Pxx, fisher=True, bias=False)
    return spectral_entropy, spectral_kurtosis

def feature_extraction(df, sensor_columns):
    features = []

    # Group the dataframe by vehicle_id
    grouped = df.groupby('vehicle_id')
    
    for vehicle_id, group in grouped:
        vehicle_features = [vehicle_id]
        
        for sensor in sensor_columns:
            sensor_data = group[sensor].values
            
            # Calculate statistical features
            mean = np.mean(sensor_data)
            median = np.median(sensor_data)
            std = np.std(sensor_data)
            var = np.var(sensor_data)
            minimum = np.min(sensor_data)
            maximum = np.max(sensor_data)
            skewness = skew(sensor_data)
            kurt = kurtosis(sensor_data)
            
            # Calculate spectral features
            spectral_entropy, spectral_kurtosis = calculate_spectral_features(sensor_data)
            
            # Append the features for this sensor to the vehicle features
            vehicle_features.extend([mean, median, std, var, minimum, maximum, skewness, kurt, spectral_entropy, spectral_kurtosis])
        
        # Append the features for this vehicle to the list
        features.append(vehicle_features)
    
    # Define column names dynamically based on the number of sensors
    column_names = ['vehicle_id']
    for sensor in sensor_columns:
        column_names.extend([f"{sensor}_mean", f"{sensor}_median", f"{sensor}_std", f"{sensor}_var", f"{sensor}_min", f"{sensor}_max", f"{sensor}_skewness", f"{sensor}_kurtosis", f"{sensor}_spectral_entropy", f"{sensor}_spectral_kurtosis"])
    
    # Create a DataFrame from the features
    feature_df = pd.DataFrame(features, columns=column_names)
    
    return feature_df

In [17]:
feature_df = feature_extraction(psd_combined_df, psd_combined_df.columns[2:])
feature_df

,vehicle_id,171_0_mean,171_0_median,171_0_std,171_0_var,171_0_min,171_0_max,171_0_skewness,171_0_kurtosis,171_0_spectral_entropy,...,397_35_mean,397_35_median,397_35_std,397_35_var,397_35_min,397_35_max,397_35_skewness,397_35_kurtosis,397_35_spectral_entropy,397_35_spectral_kurtosis
0,2,9.550564e+12,9.449861e+10,3.210733e+13,1.030881e+27,9.152348e+08,1.368028e+14,3.642203,11.520745,0.846027,...,9.728932e+04,1217.583597,2.524937e+05,6.375308e+10,6.592829,8.025761e+05,2.368818,3.627734,0.732543,2.393180
1,14,1.257518e+13,4.871166e+11,4.300053e+13,1.849046e+27,3.734490e+10,2.064344e+14,4.150941,15.784024,0.769561,...,2.854375e+05,10449.091961,1.120402e+06,1.255300e+12,174.655193,5.408409e+06,4.333233,16.871694,0.784372,3.918171
2,25,5.541963e+13,3.561477e+11,1.928583e+14,3.719434e+28,8.872380e+10,9.363080e+14,4.139557,15.939487,0.741529,...,7.636945e+05,12255.993323,2.629588e+06,6.914733e+12,2195.484598,1.281450e+07,4.175303,16.187254,0.744001,4.072708
3,27,1.698739e+13,2.381277e+10,1.063952e+14,1.131994e+28,1.019048e+09,7.442551e+14,6.651192,42.483808,0.665104,...,2.042214e+05,11130.355712,7.602224e+05,5.779380e+11,2513.091333,4.904650e+06,5.257984,28.302722,0.686764,12.638653
4,28,5.152830e+12,6.470033e+09,3.021224e+13,9.127794e+26,3.278018e+07,2.169414e+14,6.755180,44.352183,0.658970,...,1.637338e+04,132.919308,8.222028e+04,6.760174e+09,3.091989,5.789769e+05,6.330019,40.050296,0.665177,14.737901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15160,33637,1.610564e+12,1.515586e+10,5.441505e+12,2.960997e+25,4.009629e+08,2.121899e+13,3.321646,9.049819,0.973432,...,1.258414e+05,6732.107003,2.803024e+05,7.856943e+10,1783.903430,9.543760e+05,2.213600,3.253998,0.798475,0.651522
15161,33639,7.298168e+11,2.255199e+10,1.951100e+12,3.806790e+24,7.142390e+09,6.246683e+12,2.472004,4.116759,1.156760,...,1.750045e+04,1006.591017,4.385876e+04,1.923591e+09,102.287419,1.413186e+05,2.457330,4.073934,1.137420,-0.660083
15162,33640,2.874192e+12,1.934799e+10,1.061854e+13,1.127535e+26,5.978833e+08,4.527539e+13,3.728850,11.959977,0.873020,...,5.249299e+06,92967.089801,1.903836e+07,3.624590e+14,3025.468746,8.121497e+07,3.719906,11.915439,0.871095,1.279314
15163,33642,2.056574e+12,1.374424e+10,6.973499e+12,4.862968e+25,2.477302e+08,2.718979e+13,3.323564,9.057884,0.971355,...,3.887404e+04,1288.814621,8.917474e+04,7.952135e+09,107.994351,3.353173e+05,2.662378,5.843134,0.819212,0.107897


In [18]:
# label the data after feature extraction
label=train_oper_label.drop(columns=train_oper_label.columns[1:-1],axis=1)
feature_df=pd.merge(label,feature_df,on='vehicle_id')
feature_df

,vehicle_id,class_label,171_0_mean,171_0_median,171_0_std,171_0_var,171_0_min,171_0_max,171_0_skewness,171_0_kurtosis,...,397_35_mean,397_35_median,397_35_std,397_35_var,397_35_min,397_35_max,397_35_skewness,397_35_kurtosis,397_35_spectral_entropy,397_35_spectral_kurtosis
0,2,0,9.550564e+12,9.449861e+10,3.210733e+13,1.030881e+27,9.152348e+08,1.368028e+14,3.642203,11.520745,...,9.728932e+04,1217.583597,2.524937e+05,6.375308e+10,6.592829,8.025761e+05,2.368818,3.627734,0.732543,2.393180
1,14,0,1.257518e+13,4.871166e+11,4.300053e+13,1.849046e+27,3.734490e+10,2.064344e+14,4.150941,15.784024,...,2.854375e+05,10449.091961,1.120402e+06,1.255300e+12,174.655193,5.408409e+06,4.333233,16.871694,0.784372,3.918171
2,25,0,5.541963e+13,3.561477e+11,1.928583e+14,3.719434e+28,8.872380e+10,9.363080e+14,4.139557,15.939487,...,7.636945e+05,12255.993323,2.629588e+06,6.914733e+12,2195.484598,1.281450e+07,4.175303,16.187254,0.744001,4.072708
3,27,4,1.698739e+13,2.381277e+10,1.063952e+14,1.131994e+28,1.019048e+09,7.442551e+14,6.651192,42.483808,...,2.042214e+05,11130.355712,7.602224e+05,5.779380e+11,2513.091333,4.904650e+06,5.257984,28.302722,0.686764,12.638653
4,28,0,5.152830e+12,6.470033e+09,3.021224e+13,9.127794e+26,3.278018e+07,2.169414e+14,6.755180,44.352183,...,1.637338e+04,132.919308,8.222028e+04,6.760174e+09,3.091989,5.789769e+05,6.330019,40.050296,0.665177,14.737901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15160,33637,0,1.610564e+12,1.515586e+10,5.441505e+12,2.960997e+25,4.009629e+08,2.121899e+13,3.321646,9.049819,...,1.258414e+05,6732.107003,2.803024e+05,7.856943e+10,1783.903430,9.543760e+05,2.213600,3.253998,0.798475,0.651522
15161,33639,0,7.298168e+11,2.255199e+10,1.951100e+12,3.806790e+24,7.142390e+09,6.246683e+12,2.472004,4.116759,...,1.750045e+04,1006.591017,4.385876e+04,1.923591e+09,102.287419,1.413186e+05,2.457330,4.073934,1.137420,-0.660083
15162,33640,0,2.874192e+12,1.934799e+10,1.061854e+13,1.127535e+26,5.978833e+08,4.527539e+13,3.728850,11.959977,...,5.249299e+06,92967.089801,1.903836e+07,3.624590e+14,3025.468746,8.121497e+07,3.719906,11.915439,0.871095,1.279314
15163,33642,0,2.056574e+12,1.374424e+10,6.973499e+12,4.862968e+25,2.477302e+08,2.718979e+13,3.323564,9.057884,...,3.887404e+04,1288.814621,8.917474e+04,7.952135e+09,107.994351,3.353173e+05,2.662378,5.843134,0.819212,0.107897


In [19]:
# checking class distribution
feature_df["class_label"].value_counts()

0    15001
4       41
3       41
1       41
2       41
Name: class_label, dtype: int64

In [20]:
# filling missing values with 0
feature_df.fillna(0,inplace=True)

### Add specifications data

In [21]:
# read the specifications data adn convert the categorical data to numerical
train_spes=pd.read_csv("train_specifications.csv")
train_spes = train_spes.replace(to_replace=r'Cat', value='', regex=True)
train_spes=train_spes.astype(int)
train_spes

,vehicle_id,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7
0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,1
2,3,0,1,1,1,0,0,0,1
3,4,0,0,2,1,0,0,0,1
4,5,0,2,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
23545,33639,0,1,1,0,0,0,1,4
23546,33640,0,14,1,3,0,0,1,4
23547,33641,0,1,1,0,0,0,1,4
23548,33642,0,1,1,0,0,0,1,4


In [22]:
# merge the feature data with the specifications data
feature_df=pd.merge(train_spes,feature_df,on='vehicle_id')
feature_df

,vehicle_id,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7,class_label,...,397_35_mean,397_35_median,397_35_std,397_35_var,397_35_min,397_35_max,397_35_skewness,397_35_kurtosis,397_35_spectral_entropy,397_35_spectral_kurtosis
0,2,0,1,1,0,0,0,0,1,0,...,9.728932e+04,1217.583597,2.524937e+05,6.375308e+10,6.592829,8.025761e+05,2.368818,3.627734,0.732543,2.393180
1,14,0,1,1,0,0,0,0,1,0,...,2.854375e+05,10449.091961,1.120402e+06,1.255300e+12,174.655193,5.408409e+06,4.333233,16.871694,0.784372,3.918171
2,25,0,1,1,1,0,1,0,0,0,...,7.636945e+05,12255.993323,2.629588e+06,6.914733e+12,2195.484598,1.281450e+07,4.175303,16.187254,0.744001,4.072708
3,27,0,1,1,1,0,0,0,1,4,...,2.042214e+05,11130.355712,7.602224e+05,5.779380e+11,2513.091333,4.904650e+06,5.257984,28.302722,0.686764,12.638653
4,28,0,1,1,1,0,0,0,1,0,...,1.637338e+04,132.919308,8.222028e+04,6.760174e+09,3.091989,5.789769e+05,6.330019,40.050296,0.665177,14.737901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15160,33637,0,1,1,0,0,0,1,4,0,...,1.258414e+05,6732.107003,2.803024e+05,7.856943e+10,1783.903430,9.543760e+05,2.213600,3.253998,0.798475,0.651522
15161,33639,0,1,1,0,0,0,1,4,0,...,1.750045e+04,1006.591017,4.385876e+04,1.923591e+09,102.287419,1.413186e+05,2.457330,4.073934,1.137420,-0.660083
15162,33640,0,14,1,3,0,0,1,4,0,...,5.249299e+06,92967.089801,1.903836e+07,3.624590e+14,3025.468746,8.121497e+07,3.719906,11.915439,0.871095,1.279314
15163,33642,0,1,1,0,0,0,1,4,0,...,3.887404e+04,1288.814621,8.917474e+04,7.952135e+09,107.994351,3.353173e+05,2.662378,5.843134,0.819212,0.107897


### Preparing the data for training and testing

In [33]:
# data that belongs to class 0
one_0 = feature_df.loc[feature_df["class_label"]==0]

In [34]:
# test data and training data from class 0
amont_vehicle_class_0=len(one_0["vehicle_id"].unique())
normal_class=one_0.head(int(amont_vehicle_class_0*0.8))
test_class_0=one_0.tail(164)

In [35]:
# selecting the data that belongs to other classes and label them as -1
test_other_classes=feature_df.loc[feature_df["class_label"]!=0]
test_other_classes["class_label"]=-1

# label the data that belongs to class 0 as 1
train=normal_class.loc[normal_class["class_label"]==0]
train["class_label"]=1

test_class_0["class_label"]=1

test=pd.concat([test_class_0,test_other_classes])

In [36]:
# split the data into features and labels
train_x = train.drop('class_label', axis=1)
test_x = test.drop('class_label', axis=1)
test_y = test['class_label']

In [37]:
# standard the data using standard scaler other variables to test and train
scaler = StandardScaler()
train_x_stand = scaler.fit_transform(train_x)
test_x_stand = scaler.transform(test_x)

### Modeling and evaluating 

In [38]:
# Initialize the OneClassSVM model
one_class_svm_model = OneClassSVM(kernel='poly', nu=0.01)  
one_class_svm_model_std = OneClassSVM(kernel='poly', nu=0.01)

# Train the model
one_class_svm_model.fit(train_x)
one_class_svm_model_std.fit(train_x_stand)

# Note that OneClassSVM will output 1 for inliers and -1 for outliers
y_pred_one_class_svm = one_class_svm_model.predict(test_x)
y_pred_one_class_svm_std = one_class_svm_model_std.predict(test_x_stand)

# Calculate confusion matrix
conf_mat_one_class_svm = confusion_matrix(test_y, y_pred_one_class_svm)
conf_mat_one_class_svm_std = confusion_matrix(test_y, y_pred_one_class_svm_std)

# Generate the classification report
report_one_class_svm = classification_report(test_y, y_pred_one_class_svm)
report_one_class_svm_std = classification_report(test_y, y_pred_one_class_svm_std)

# Evaluate the model
print("OneClassSVM Classification Report:")
print(report_one_class_svm)
print("OneClassSVM Classification Report std:")
print(report_one_class_svm_std)
print("OneClassSVM Confusion Matrix std:")
print(conf_mat_one_class_svm_std)
print("OneClassSVM Confusion Matrix:")
print(conf_mat_one_class_svm)

OneClassSVM Classification Report:
              precision    recall  f1-score   support

          -1       0.46      0.45      0.45       164
           1       0.46      0.48      0.47       164

    accuracy                           0.46       328
   macro avg       0.46      0.46      0.46       328
weighted avg       0.46      0.46      0.46       328

OneClassSVM Classification Report std:
              precision    recall  f1-score   support

          -1       0.33      0.05      0.09       164
           1       0.49      0.89      0.63       164

    accuracy                           0.47       328
   macro avg       0.41      0.47      0.36       328
weighted avg       0.41      0.47      0.36       328

OneClassSVM Confusion Matrix std:
[[  9 155]
 [ 18 146]]
OneClassSVM Confusion Matrix:
[[73 91]
 [85 79]]


In [44]:
# Initialize and fit the IsolationForest model
iso_forest_model_std = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
iso_forest_model_std.fit(train_x_stand)

y_pred_iso_forest_std = iso_forest_model_std.predict(test_x_stand)

conf_mat_iso_forest_std = confusion_matrix(test_y, y_pred_iso_forest_std)
report_iso_forest_std = classification_report(test_y, y_pred_iso_forest_std)

print("Isolation Forest Classification Report std:")
print(report_iso_forest_std)

print("Isolation Forest Confusion Matrix std:")
print(conf_mat_iso_forest_std)


Isolation Forest Classification Report std:
              precision    recall  f1-score   support

          -1       0.69      0.05      0.10       164
           1       0.51      0.98      0.67       164

    accuracy                           0.52       328
   macro avg       0.60      0.52      0.38       328
weighted avg       0.60      0.52      0.38       328

Isolation Forest Confusion Matrix std:
[[  9 155]
 [  4 160]]


In [40]:
# Initialize and fit the LOF model
lof_model = LocalOutlierFactor(n_neighbors=20, contamination='auto', novelty=True)
lof_model.fit(train_x)

# Initialize and fit the LOF with std data
lof_model_std = LocalOutlierFactor(n_neighbors=20, contamination='auto', novelty=True)
lof_model_std.fit(train_x_stand)

# Predict the labels of the test set
y_pred_lof = lof_model.predict(test_x)
y_pred_lof_std = lof_model_std.predict(test_x_stand)

# Evaluate the model
conf_mat_lof = confusion_matrix(test_y, y_pred_lof)
report_lof = classification_report(test_y, y_pred_lof)
conf_mat_lof_std = confusion_matrix(test_y, y_pred_lof_std)
report_lof_std = classification_report(test_y, y_pred_lof_std)

print("Local Outlier Factor (LOF) Classification Report:")
print(report_lof)
print("Local Outlier Factor (LOF) Classification Report std:")
print(report_lof_std)
print("Local Outlier Factor (LOF) Confusion Matrix:")
print(conf_mat_lof)
print("Local Outlier Factor (LOF) Confusion Matrix std:")
print(conf_mat_lof_std)


Local Outlier Factor (LOF) Classification Report:
              precision    recall  f1-score   support

          -1       0.47      0.05      0.10       164
           1       0.50      0.94      0.65       164

    accuracy                           0.50       328
   macro avg       0.49      0.50      0.37       328
weighted avg       0.49      0.50      0.37       328

Local Outlier Factor (LOF) Classification Report std:
              precision    recall  f1-score   support

          -1       0.49      0.20      0.28       164
           1       0.50      0.80      0.61       164

    accuracy                           0.50       328
   macro avg       0.50      0.50      0.45       328
weighted avg       0.50      0.50      0.45       328

Local Outlier Factor (LOF) Confusion Matrix:
[[  9 155]
 [ 10 154]]
Local Outlier Factor (LOF) Confusion Matrix std:
[[ 32 132]
 [ 33 131]]


### Modling and testing different parameter values and computing the AUC

In [45]:
contamination_values = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]  # range of contamination values
n_neighbors_values = [ 20, 25, 30, 35, 40, 50, 100]  # range of n_estimators values

test_y_binary = np.where(test_y == -1, 1, 0)

for cont, n_est in zip(contamination_values, n_neighbors_values):
    # Initialize and fit the LOF Forest model
    lof_model = LocalOutlierFactor(n_neighbors=n_est, contamination=cont, novelty=True)
    lof_model.fit(train_x)

    scores = lof_model.score_samples(test_x)
    
    # Calculate AUC
    auc = roc_auc_score(test_y_binary, -scores)
    
    # Predict the labels of the test set
    y_pred = lof_model.predict(test_x)
    y_pred_binary = np.where(y_pred == -1, 1, 0)
    
    # Print model configuration, AUC, and evaluation metrics
    print(f"Contamination: {cont}, n_estimators: {n_est}")
    print(f"AUC: {auc}")
    print("Confusion Matrix:")
    print(confusion_matrix(test_y_binary, y_pred_binary))
    print("Classification Report:")
    print(classification_report(test_y_binary, y_pred_binary))
    print("----------")


Contamination: 0.01, n_estimators: 20
AUC: 0.4243753718024985
Confusion Matrix:
[[162   2]
 [161   3]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.99      0.67       164
           1       0.60      0.02      0.04       164

    accuracy                           0.50       328
   macro avg       0.55      0.50      0.35       328
weighted avg       0.55      0.50      0.35       328

----------
Contamination: 0.05, n_estimators: 25
AUC: 0.4282421177870316
Confusion Matrix:
[[151  13]
 [155   9]]
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.92      0.64       164
           1       0.41      0.05      0.10       164

    accuracy                           0.49       328
   macro avg       0.45      0.49      0.37       328
weighted avg       0.45      0.49      0.37       328

----------
Contamination: 0.1, n_estimators: 30
AUC: 0.4307703747769185
Confusion M

In [46]:
contamination_values = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]  # range of contamination values
n_neighbors_values = [ 20, 25, 30, 35, 40, 50, 100]  # range of n_estimators values

test_y_binary = np.where(test_y == -1, 1, 0)

for cont, n_est in zip(contamination_values, n_neighbors_values):
    # Initialize and fit the Isolation Forest model
    iso_forest_model = IsolationForest(n_estimators=n_est, contamination=cont, random_state=42)
    iso_forest_model.fit(train_x_stand)
    scores = iso_forest_model.score_samples(test_x_stand)
    
    # Calculate AUC
    auc = roc_auc_score(test_y_binary, -scores)
    
    # Predict the labels of the test set
    y_pred = iso_forest_model.predict(test_x_stand)
    y_pred_binary = np.where(y_pred == -1, 1, 0)
    
    # Print model configuration, AUC, and evaluation metrics
    print(f"Contamination: {cont}, n_estimators: {n_est}")
    print(f"AUC: {auc}")
    print("Confusion Matrix:")
    print(confusion_matrix(test_y_binary, y_pred_binary))
    print("Classification Report:")
    print(classification_report(test_y_binary, y_pred_binary))
    print("----------")


Contamination: 0.01, n_estimators: 20
AUC: 0.5675565139797739
Confusion Matrix:
[[163   1]
 [161   3]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.99      0.67       164
           1       0.75      0.02      0.04       164

    accuracy                           0.51       328
   macro avg       0.63      0.51      0.35       328
weighted avg       0.63      0.51      0.35       328

----------
Contamination: 0.05, n_estimators: 25
AUC: 0.5568114217727543
Confusion Matrix:
[[159   5]
 [156   8]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.97      0.66       164
           1       0.62      0.05      0.09       164

    accuracy                           0.51       328
   macro avg       0.56      0.51      0.38       328
weighted avg       0.56      0.51      0.38       328

----------
Contamination: 0.1, n_estimators: 30
AUC: 0.5574063057703749
Confusion M